# Init

In [1]:
ON_COLAB = True
if ON_COLAB:
    import os
    os.chdir('/content/')
    !pip install -q datasets accelerate torchvision youtube_dl evaluate
    !git clone https://github.com/linv24/multimodal-commonsense.git
    os.chdir('/content/multimodal-commonsense/dev/scripts')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
Cloning into 'multimodal-commonsense'...
remote: Enumerating objects: 2984, done.
remote: Counting objects: 100% (1849/1849), done.
remote: Compressing objects: 100% (1819/1819), done.
remote: Total 2984 (delta 39), reused 1807 (delta 30), pack-reused 1135
Receiving objects: 100% (2984/2984), 290.78 MiB | 33.72 MiB/s, done.
Resolving deltas: 100% (248/248), done.
Updating files: 100% (1907/1907), done.


In [2]:
!pip install -q evaluate

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import transformers
from transformers import (AutoTokenizer, FlavaModel, BertTokenizer,
                          FlavaProcessor, AutoModelForMultipleChoice, Trainer,
                          TrainingArguments)
from transformers.modeling_outputs import MultipleChoiceModelOutput
from datasets import Dataset, load_dataset
import torchvision
from torchvision.io import read_image
import pandas as pd
import json
import evaluate
import numpy as np

import data_util

# Testing

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
s1 = 'i have cake and i am hungry, so'
s2 = 'i will eat a slice of cake'

tokens = tokenizer(s1, s2)
tokens

{'input_ids': [101, 1045, 2031, 9850, 1998, 1045, 2572, 7501, 1010, 2061, 102, 1045, 2097, 4521, 1037, 14704, 1997, 9850, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [29]:
# default flava model
model = FlavaModel.from_pretrained('facebook/flava-full')
tokenizer = AutoTokenizer.from_pretrained('facebook/flava-full')

`text_config_dict` is provided which will be used to initialize `FlavaTextConfig`. The value `text_config["id2label"]` will be overriden.
`multimodal_config_dict` is provided which will be used to initialize `FlavaMultimodalConfig`. The value `multimodal_config["id2label"]` will be overriden.
`image_codebook_config_dict` is provided which will be used to initialize `FlavaImageCodebookConfig`. The value `image_codebook_config["id2label"]` will be overriden.


In [ ]:
# how tokenizer combines text and text_pair as lists of strings
tokens = tokenizer(['sent1 part1', 'sent2 part2'], ['sent1 part2', 'sent2 part2'])
tokenizer.decode(tokens['input_ids'][0])

'[CLS] sent1 part1 [SEP] sent1 part2 [SEP]'

In [ ]:
# text only input
text = 'a photo of a dog'
inputs = tokenizer(text=[text], return_tensors='pt', padding='max_length', max_length=77)
text_embedding = model.get_text_features(**inputs)

print(f'{inputs=}')
print(f'{text_embedding=}') # length of max_length

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/modeling_utils.py:900: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


inputs={'input_ids': tensor([[ 101, 1037, 6302, 1997, 1037, 3899,  102,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
out = model(**inputs)
out.keys()

odict_keys(['text_embeddings', 'text_output'])

In [30]:
# multimodal input
processor = FlavaProcessor.from_pretrained("facebook/flava-full")

source_id = '-1IBHYS3L-Y'
text = 'Then, the man writes over the snow covering the window of a car, and a woman wearing winter clothes smiles. then, the man adds wax to the windshield and cuts it.' # first training example context + ending 0
image = read_image(f'../data/hellaswag_images/{source_id}.png') # shape = torch.Size([3, 144, 192])

inputs = processor(
    text=[text],
    images=[image],
    return_tensors='pt',
    padding='max_length',
    max_length=128,
)
print(f'{inputs.keys()=}')

outputs = model(**inputs)
image_embeddings = outputs.image_embeddings # Batch size X (Number of image patches + 1) x Hidden size => 2 X 197 X 768
text_embeddings = outputs.text_embeddings # Batch size X (Text sequence length + 1) X Hidden size => 2 X 77 X 768
multimodal_embeddings = outputs.multimodal_embeddings # Batch size X (Number of image patches + Text Sequence Length + 3) X Hidden size => 2 X 275 x 768

outputs.text_output.keys()

inputs.keys()=dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values'])


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:907: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states'])

In [ ]:
outputs.keys()

odict_keys(['image_embeddings', 'image_output', 'text_embeddings', 'text_output', 'multimodal_embeddings', 'multimodal_output'])

# Data processing

In [4]:
processor = FlavaProcessor.from_pretrained("facebook/flava-full")
tokenizer = AutoTokenizer.from_pretrained('facebook/flava-full')

preprocessor_config.json:   0%|          | 0.00/881 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
# generate encoded ds from jsonl


def preprocess_data(batch): # garbage testing preprocessing
    # each prompt in batch is converted to a list of strings for ctx_a and
    # ctx_b + ending
    # duplicate ctx_a 4 times, one for each ending
    first_sentences = [[ctx_a] * 4 for ctx_a in batch['ctx_a']]
    # print(f'{batch["ctx_a"]=}')
    # print(f'{first_sentences=}')
    # print(f'{len(first_sentences)=}')

    # ctx_b + each ending as a list
    second_sentences = [[f'{ctx_b} {ending}' for ending in batch['endings'][ix]]
                        for ix, ctx_b in enumerate(batch['ctx_b'])]
    # print(f'{batch["ctx_b"]=}')
    # print(f'{second_sentences=}')
    # print(f'{len(second_sentences)=}')

    # flatten contexts and endings
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])
    # print()
    # print(f'{first_sentences=}')
    # print(f'{second_sentences=}')
    # print(f'{len(first_sentences)=}')
    # print(f'{len(second_sentences)=}')

    encoding = processor(text=list(zip(first_sentences, second_sentences)),
                         images=None,
                         return_tensors='pt',
                         truncation=True,
                         max_length=128,
                         padding='max_length',)
    # encoding['label'] = batch['label']
    # print(f'{encoding.keys()}')
    # print(f'{len(encoding["input_ids"])=}')
    # print(f'{list(encoding.items())[0]=}')
    ret = {k: [v[i : i + 4] for i in range(0, len(v), 4)] for k, v in encoding.items()}
    if 'label' in batch:
        ret['labels'] = batch['label']
    # print(f'{ret.keys()=}')
    # print(f'{len(ret["input_ids"])=}')
    # print(f'{len(ret["label"])=}')
    # print(f'{ret["input_ids"][0]=}')
    # print(f'{ret["label"][0]=}')

    return ret

# ds = data_util.jsonl2ds('../data/jsonl/tiny_hellaswag_train.jsonl')

# encoded_ds = ds.map(preprocess_data, batched=True)
# print(encoded_ds)

# for i in range(2):
#     for j in range(4):
#         print(tokenizer.decode(encoded_ds['input_ids'][i][j]))
#     print(encoded_ds['label'][i])

# Trainer

In [46]:
output_dir = './checkpoint'
logging_dir = './logging'
batch_size = 8
learning_rate = 2e-5
num_train_epochs = 10

In [47]:
train_ds = data_util.jsonl2ds('../data/jsonl/sampled_hellaswag_train.jsonl')
train_ds = train_ds.map(preprocess_data, batched=True,
                        remove_columns=['ind', 'activity_label', 'ctx_a', 'ctx_b', 'ctx', 'split', 'split_type', 'endings', 'source_id'])

split_ds = train_ds.train_test_split(test_size=0.2)
train_ds, val_ds = split_ds['train'], split_ds['test']

print(train_ds)

# train_dl = DataLoader(train_ds, batch_size=batch_size)
# val_dl = DataLoader(val_ds, batch_size=batch_size)

test_ds = data_util.jsonl2ds('../data/jsonl/sampled_hellaswag_val.jsonl')
test_ds = test_ds.map(preprocess_data, batched=True,
                      remove_columns=['ind', 'activity_label', 'ctx_a', 'ctx_b', 'ctx', 'split', 'split_type', 'endings', 'source_id'])

Map:   0%|          | 0/1474 [00:00<?, ? examples/s]

Dataset({
    features: ['label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1179
})


Map:   0%|          | 0/325 [00:00<?, ? examples/s]

In [48]:
training_args = TrainingArguments(
    output_dir=output_dir,
    logging_dir=logging_dir,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    optim="adamw_torch",
)

In [49]:
accuracy = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# FLAVA

In [62]:
class FlavaForMultipleChoice(nn.Module):
    def __init__(self, hidden_dropout_prob=0.1, hidden_size=768):
        super().__init__()

        self.flava = FlavaModel.from_pretrained('facebook/flava-full')
        self.dropout = nn.Dropout(hidden_dropout_prob)
        self.classifier = nn.Linear(hidden_size, 1)

    def forward(self,
                input_ids=None,
                token_type_ids=None,
                attention_mask=None,
                labels=None):
        if type(input_ids) == list:
            input_ids = torch.tensor(input_ids, dtype=torch.long)
        if type(token_type_ids) == list:
            token_type_ids = torch.tensor(token_type_ids, dtype=torch.long)
        if type(attention_mask) == list:
            attention_mask = torch.tensor(attention_mask, dtype=torch.long)
        if type(labels) == list:
            labels = torch.tensor(labels, dtype=torch.long)

        num_choices = input_ids.shape[1]
        print(f'{num_choices=}')

        flat_input_ids = input_ids.view(-1, input_ids.size(-1))
        flat_token_type_ids = token_type_ids.view(-1, token_type_ids.size(-1))
        flat_attention_mask = attention_mask.view(-1, attention_mask.size(-1))

        outputs = self.flava(
            input_ids=flat_input_ids,
            token_type_ids=flat_token_type_ids,
            attention_mask=flat_attention_mask,
        )

        print(outputs.text_embeddings.shape)
        # text_output.keys = odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states'])
        text_pooled_output = outputs.text_output['pooler_output']
        text_pooled_output = self.dropout(text_pooled_output)
        logits = self.classifier(text_pooled_output)
        reshaped_logits = logits.view(-1, num_choices)
        preds = torch.argmax(torch.softmax(reshaped_logits, dim=1))

        # print(f'{text_pooled_output.shape=}')
        # print(f'{logits.shape=}')

        # print(f'{text_pooled_output[0]=}')
        # print(f'{text_pooled_output[1]=}')

        # print(f'{logits=}')
        # print(f'{reshaped_logits=}')
        # print(f'{reshaped_logits.shape=}')
        # print(f'{torch.softmax(reshaped_logits, dim=1)=}')
        # print(f'{preds.shape=}, {labels.shape=}')

        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(reshaped_logits, labels) # debug

        return MultipleChoiceModelOutput(
            loss=loss,
            logits=reshaped_logits,
        )

        # note:
        # inputs.keys()=dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values'])
        # odict_keys(['image_embeddings', 'image_output', 'text_embeddings', 'text_output', 'multimodal_embeddings', 'multimodal_output'])


flava_model = FlavaForMultipleChoice()

examples = train_ds.select(range(4))

for i in range(2):
    for j in range(4):
        print(tokenizer.decode(examples['input_ids'][i][j]))

out = flava_model(
    input_ids=examples['input_ids'],
    token_type_ids=examples['token_type_ids'],
    attention_mask=examples['attention_mask'],
    labels=examples['label']
)
out


`text_config_dict` is provided which will be used to initialize `FlavaTextConfig`. The value `text_config["id2label"]` will be overriden.
`multimodal_config_dict` is provided which will be used to initialize `FlavaMultimodalConfig`. The value `multimodal_config["id2label"]` will be overriden.
`image_codebook_config_dict` is provided which will be used to initialize `FlavaImageCodebookConfig`. The value `image_codebook_config["id2label"]` will be overriden.


[CLS] a woman is seen speaking to the camera and pans around an old table. [SEP] the girl plays a flute while the camera follows her all around. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] a woman is seen speaking to the camera and pans around an old table. [SEP] the girl begins vacuuming and putting her feet into her tennis shoes. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:907: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


torch.Size([16, 128, 768])
self.classifier.weight=Parameter containing:
tensor([[ 2.7624e-02,  7.6531e-04,  2.8311e-02, -2.4458e-02, -5.9771e-03,
         -1.4030e-02,  3.0990e-02, -3.4468e-02,  2.7497e-02,  3.2286e-02,
          6.6507e-03,  3.2401e-02,  2.8663e-02,  2.7976e-02, -1.6366e-02,
         -2.2360e-02, -3.4484e-02,  3.3698e-02,  8.3199e-03,  2.3519e-03,
          2.1560e-02, -2.2932e-02,  1.3693e-02,  2.4584e-02,  2.2865e-02,
          1.4104e-02, -1.4485e-02,  1.4876e-02,  2.7374e-02, -3.3580e-02,
          2.1863e-02,  2.5100e-02,  1.4164e-02,  2.4950e-02, -3.5848e-02,
          3.3835e-02, -1.6343e-02,  7.1228e-04,  1.1200e-02,  1.3996e-02,
          3.2836e-02,  4.6183e-03, -7.5606e-03,  1.5499e-02, -2.5599e-02,
         -1.3106e-02, -2.1333e-02, -2.2605e-02, -1.9182e-02,  9.6316e-03,
         -3.0815e-02,  4.3632e-03,  1.0450e-02, -3.4848e-02,  3.2194e-02,
          3.0246e-02,  5.6951e-03,  1.3245e-02,  7.1969e-03, -1.2346e-03,
         -2.4834e-02, -2.6871e-02, -8.67

ValueError: ignored

In [51]:
flava_model = FlavaForMultipleChoice()

examples = train_ds.select(range(4))

# for i in range(2):
#     for j in range(4):
#         print(tokenizer.decode(examples['input_ids'][i][j]))

out = flava_model(
    input_ids=examples['input_ids'],
    token_type_ids=examples['token_type_ids'],
    attention_mask=examples['attention_mask'],
    labels=examples['label']
)
out

`text_config_dict` is provided which will be used to initialize `FlavaTextConfig`. The value `text_config["id2label"]` will be overriden.
`multimodal_config_dict` is provided which will be used to initialize `FlavaMultimodalConfig`. The value `multimodal_config["id2label"]` will be overriden.
`image_codebook_config_dict` is provided which will be used to initialize `FlavaImageCodebookConfig`. The value `image_codebook_config["id2label"]` will be overriden.


MultipleChoiceModelOutput(loss=tensor(1.3863, grad_fn=<NllLossBackward0>), logits=tensor([[0.0145, 0.0145, 0.0145, 0.0145],
        [0.0145, 0.0145, 0.0145, 0.0145],
        [0.0145, 0.0145, 0.0145, 0.0145],
        [0.0145, 0.0145, 0.0145, 0.0145]], grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [52]:
# model = nn.Sequential(
#     nn.Linear(10, 32),
#     nn.ReLU(),
#     nn.Linear(32, 1)
# )

flava_trainer = Trainer(model=flava_model,
                        args=training_args,
                        train_dataset=train_ds,
                        eval_dataset=val_ds,
                        compute_metrics=compute_metrics)

In [53]:
flava_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.355400,1.329681,0.410169
2,0.974900,1.323240,0.413559
3,0.488300,1.521640,0.423729
4,0.216700,1.748638,0.389831


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:907: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:907: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:907: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:907: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:907: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


KeyboardInterrupt: ignored

In [ ]:
eval_out = flava_trainer.evaluate(test_ds)
eval_out

# BERT

In [ ]:
bert_model = AutoModelForMultipleChoice.from_pretrained('bert-base-uncased')

In [ ]:
bert_trainer = Trainer(model=bert_model,
                       args=training_args,
                       train_dataset=train_ds,
                       eval_dataset=val_ds,
                       compute_metrics=compute_metrics
                       )

In [ ]:
bert_trainer.train()